## 환경 준비

In [1]:
!wget https://raw.githubusercontent.com/NLP-kr/tensorflow-ml-nlp-tf2/master/requirements.txt -O requirements.txt
!pip install -r requirements.txt
!pip install tensorflow==2.2.0


--2021-10-04 14:41:12--  https://raw.githubusercontent.com/NLP-kr/tensorflow-ml-nlp-tf2/master/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 610 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]     610  --.-KB/s    in 0s      

2021-10-04 14:41:12 (26.0 MB/s) - ‘requirements.txt’ saved [610/610]

     |████████████████████████████████| 17.3 MB 25 kB/s 
     |████████████████████████████████| 1.4 MB 37.5 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 769 kB 46.8 MB/s 
     |████████████████████████████████| 1.0 MB 46.7 MB/s 
     |████████████████████████████████| 46.9 MB 44 kB/s 
     |████████████████████████████████| 252 kB 47.0 MB/s 
     |█

## 데이터 다운로드

In [2]:
!mkdir -p data_in/KOR/naver_movie
!wget https://raw.githubusercontent.com/NLP-kr/tensorflow-ml-nlp-tf2/master/7.PRETRAIN_METHOD/data_in/KOR/naver_movie/ratings_train.txt \
              -O data_in/KOR/naver_movie/ratings_train.txt
!wget https://raw.githubusercontent.com/NLP-kr/tensorflow-ml-nlp-tf2/master/7.PRETRAIN_METHOD/data_in/KOR/naver_movie/ratings_test.txt \
              -O data_in/KOR/naver_movie/ratings_test.txt

--2021-10-04 14:43:24--  https://raw.githubusercontent.com/NLP-kr/tensorflow-ml-nlp-tf2/master/7.PRETRAIN_METHOD/data_in/KOR/naver_movie/ratings_train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: ‘data_in/KOR/naver_movie/ratings_train.txt’

data_in/KOR/naver_m 100%[===================>]  13.95M  --.-KB/s    in 0.1s    

2021-10-04 14:43:24 (103 MB/s) - ‘data_in/KOR/naver_movie/ratings_train.txt’ saved [14628807/14628807]

--2021-10-04 14:43:25--  https://raw.githubusercontent.com/NLP-kr/tensorflow-ml-nlp-tf2/master/7.PRETRAIN_METHOD/data_in/KOR/naver_movie/ratings_test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.gi

In [3]:
import os
import tensorflow as tf
from transformers import TFGPT2LMHeadModel

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import gluonnlp as nlp
from gluonnlp.data import SentencepieceTokenizer

import pandas as pd
import matplotlib.pyplot as plt

import numpy as np
import re
from random import sample

아레 실행 커멘드는 gpt_ckpt 폴더가 있지 않은 경우에만 실행해주세요.

In [4]:
import wget
import zipfile

wget.download('https://github.com/NLP-kr/tensorflow-ml-nlp-tf2/releases/download/v1.0/gpt_ckpt.zip')

with zipfile.ZipFile('gpt_ckpt.zip') as z:
    z.extractall()

In [5]:
# 시각화

def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string], '')
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()

In [6]:
SEED_NUM = 1234
tf.random.set_seed(SEED_NUM)
np.random.seed(SEED_NUM)

## 퓨샷 러닝을 위한 네이버 영화 리뷰 모델 구성


In [22]:
TOKENIZER_PATH = './gpt_ckpt/gpt2_kor_tokenizer.spiece'

tokenizer = SentencepieceTokenizer(TOKENIZER_PATH)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(TOKENIZER_PATH,
                                               mask_token=None,
                                               sep_token='<unused0>',
                                               cls_token=None,
                                               unknown_token='<unk>',
                                               padding_token='<pad>',
                                               bos_token='<s>',
                                               eos_token='</s>')

In [8]:
class TFGPT2FewshotClassifier(tf.keras.Model):
    def __init__(self, dir_path):
        super(TFGPT2FewshotClassifier, self).__init__()
        self.gpt2 = TFGPT2LMHeadModel.from_pretrained(dir_path)
        
    def call(self, inputs):
        outputs = self.gpt2({'input_ids': inputs})[0][:, -1, :]

        return outputs

In [12]:
BASE_MODEL_PATH = './gpt_ckpt'
cls_model = TFGPT2FewshotClassifier(dir_path=BASE_MODEL_PATH)

All model checkpoint weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the model checkpoint at ./gpt_ckpt.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


## 네이버 영화 리뷰 데이터 퓨샷 러닝 데이터 구성

In [9]:
# 데이터 전처리 준비
DATA_IN_PATH = './data_in/KOR'
DATA_OUT_PATH = "./data_out/KOR"

DATA_TRAIN_PATH = os.path.join(DATA_IN_PATH, "naver_movie", "ratings_train.txt")
DATA_TEST_PATH = os.path.join(DATA_IN_PATH, "naver_movie", "ratings_test.txt")

train_data = pd.read_csv(DATA_TRAIN_PATH, header = 0, delimiter = '\t', quoting = 3)
train_data = train_data.dropna()

In [10]:
print('데이터 positive 라벨: ', tokenizer('긍정'))
print('데이터 negative 라벨: ', tokenizer('부정'))

데이터 positive 라벨:  ['▁긍정']
데이터 negative 라벨:  ['▁부정']


In [11]:
print('학습 예시 케이스 구조: ', tokenizer('문장: 오늘 기분이 좋아\n감정: 긍정\n'))

학습 예시 케이스 구조:  ['▁문장', ':', '▁오늘', '▁기분이', '▁좋아', '▁감정', ':', '▁긍정']


In [18]:
print('gpt2 최대 토큰 길이: ', cls_model.gpt2.config.n_ctx)

gpt2 최대 토큰 길이:  1024


In [24]:
sent_lens = [len(tokenizer(s)) for s in train_data['document']]

print('Few shot 케이스 토큰 평균 길이: ', np.mean(sent_lens))
print('Few shot 케이스 토큰 최대 길이: ', np.max(sent_lens))
print('Few shot 케이스 토큰 길이 표준편차: ',np.std(sent_lens))
print('Few shot 케이스 토큰 길이 80 퍼센타일: ',np.percentile(sent_lens, 80))

Few shot 케이스 토큰 평균 길이:  17.206493549784994
Few shot 케이스 토큰 최대 길이:  141
Few shot 케이스 토큰 길이 표준편차:  13.920690644605873
Few shot 케이스 토큰 길이 80 퍼센타일:  24.0


In [25]:
train_fewshot_data =[]

for train_sent, train_label in train_data[['document', 'label']].values:
   tokens = vocab[tokenizer(train_sent)]

   if len(tokens) <= 25:
       train_fewshot_data.append((train_sent, train_label))

## 네이버 영화 리뷰 GPT2 퓨샷 러닝 및 평가

In [26]:
test_data = pd.read_csv(DATA_TEST_PATH, header=0, delimiter='\t', quoting=3)
test_data = test_data.dropna()
test_data.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [28]:
sample_size = 5000

train_fewshot_samples = []

for _ in range(sample_size):
    fewshot_examples = sample(train_fewshot_data, 30)
    train_fewshot_samples.append(fewshot_examples)

if sample_size < len(test_data['id']):
    test_data = test_data.sample(sample_size, random_state=SEED_NUM)

In [32]:
def build_prompt_text(sent):
    return "문장: " + sent + '\n감정: '

def clean_text(sent):
    sent_clean = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", sent)
    return sent_clean

real_labels = []
pred_tokens = []

for i, (test_sent, test_label) in enumerate(test_data[['document','label']].values):
    tokens = [vocab[vocab.bos_token]]

    for ex in train_fewshot_samples[i]:
        example_text, example_label = ex
        cleaned_example_text = clean_text(example_text)
        appended_prompt_example_text = build_prompt_text(cleaned_example_text)
        appended_prompt_example_text += '긍정' if example_label == 1 else '부정' + '\n'

        tokens += vocab[tokenizer(appended_prompt_example_text)]

    cleaned_sent = clean_text(test_sent)
    appended_prompt_sent = build_prompt_text(cleaned_sent)
    test_tokens = vocab[tokenizer(appended_prompt_sent)]

    tokens += test_tokens

    pred = tf.argmax(cls_model(np.array([tokens], dtype=np.int64)), axis=-1).numpy()
    label = vocab[tokenizer('긍정')] if test_label == 1 else vocab[tokenizer('부정')]

    pred_tokens.append(pred[0])
    real_labels.append(label[0])

In [33]:
accuracy_match = [p == t for p, t in zip(pred_tokens, real_labels)]
accuracy = len([m for m in accuracy_match if m]) / len(real_labels)

print(accuracy)

0.5608


In [34]:
def build_prompt_text(sent):
    return "감정 분석 문장: " + sent + '\n결과: '

real_labels = []
pred_tokens = []


for i, (test_sent, test_label) in enumerate(test_data[['document','label']].values):
    tokens = [vocab[vocab.bos_token]]

    for ex in train_fewshot_samples[i]:
        example_text, example_label = ex
        cleaned_example_text = clean_text(example_text)
        appended_prompt_example_text = build_prompt_text(cleaned_example_text)
        appended_prompt_example_text += '긍정' if example_label == 1 else '부정' + '\n'

        tokens += vocab[tokenizer(appended_prompt_example_text)]

    cleaned_sent = clean_text(test_sent)
    appended_prompt_sent = build_prompt_text(cleaned_sent)
    test_tokens = vocab[tokenizer(appended_prompt_sent)]

    tokens += test_tokens

    pred = tf.argmax(cls_model(np.array([tokens], dtype=np.int64)), axis=-1).numpy()
    label = vocab[tokenizer('긍정')] if test_label == 1 else vocab[tokenizer('부정')]

    pred_tokens.append(pred[0])
    real_labels.append(label[0])

In [35]:
accuracy_match = [p == t for p, t in zip(pred_tokens, real_labels)]
accuracy = len([m for m in accuracy_match if m]) / len(real_labels)

print(accuracy)

0.6024